# 🕸️ GNN (Graph Attention Network) - Оптимізоване навчання

**Модель:** Graph Neural Network with Attention Mechanism  
**Параметри:** ~300,000  
**Характеристики:** Природньо моделює Sudoku структуру, attention weights  

---

## ⚖️ Стандартизовані параметри (однакові для всіх моделей):
- **Epochs:** 40 ✅
- **Batch size:** 64 ✅
- **Learning rate:** 0.001 ✅
- **Weight decay:** 1e-5 ✅
- **Optimizer:** Adam ✅
- **Scheduler:** ReduceLROnPlateau ✅
- **Gradient clipping:** 1.0 (ОБОВ'ЯЗКОВО, в train.py) ✅
- **Dataset:** 1M+ puzzles (весь датасет) ✅

## 🔧 Архітектурні параметри (специфічні для GNN):
- **Hidden channels:** 128
- **GNN layers:** 8
- **Attention heads:** 4
- **Dropout:** 0.1

**Очікуваний результат:** 93-95% cell accuracy, 65-75% board accuracy  
**Теоретично найкраща модель для Sudoku!**

## 📦 Крок 1: Налаштування середовища

In [ ]:
# Підключення Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

print("✅ Google Drive mounted!")

In [ ]:
# Встановлення залежностей (ВАЖЛИВО для GNN!)
print("📦 Installing dependencies for GNN...")
print("⚠️ This may take 5-10 minutes!\n")

# PyTorch (стабільна версія)
!pip uninstall torch torchvision torchaudio -y -q
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121 -q

# PyTorch Geometric (ОБОВ'ЯЗКОВО для GNN)
print("Installing PyTorch Geometric...")
!pip install torch_geometric -q
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html -q

# Додаткові бібліотеки
!pip install tqdm pandas numpy matplotlib scikit-learn -q

print("\n✅ All dependencies installed!")

In [ ]:
# Перевірка GPU та PyTorch Geometric
import torch
try:
    import torch_geometric
    print(f"✅ PyTorch Geometric version: {torch_geometric.__version__}")
except ImportError:
    print("❌ PyTorch Geometric not installed!")

if torch.cuda.is_available():
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU not available, will use CPU (training will be SLOW!)")

## 📂 Крок 2: Підготовка проєкту

In [ ]:
# Шлях до проєкту (змініть на свій!)
PROJECT_PATH = '/content/drive/MyDrive/Diplom/backend'

# Перехід у папку проєкту
os.chdir(PROJECT_PATH)
print(f"📂 Working directory: {os.getcwd()}")

# Перевірка наявності файлів
required_files = ['train.py', 'dataset.py', 'models/__init__.py', 'models/gnn_model.py', 'data/sudoku.csv']
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - NOT FOUND!")

## 🎓 Крок 3: Навчання GNN

### Оптимальні параметри для максимального результату

⚠️ **ВАЖЛИВО:**
- GNN потребує більше часу на навчання (~60 хв на GPU)
- Gradient clipping ОБОВ'ЯЗКОВИЙ (вже в train.py)
- Нижчий batch size через graph processing

In [ ]:
# === НАВЧАННЯ GNN ===
# ⚖️ СТАНДАРТИЗОВАНІ ПАРАМЕТРИ (однакові для всіх моделей)

!python train.py \
    --model gnn \
    --epochs 40 \
    --batch-size 64 \
    --lr 0.001 \
    --hidden-channels 128 \
    --num-gnn-layers 8 \
    --device cuda \
    --weight-decay 1e-5 \
    --limit 1000000 \
    --save-dir weights

# 🎯 ФІКСОВАНІ ПАРАМЕТРИ (для чесного порівняння):
# --epochs 40                  : ОДНАКОВЕ для всіх моделей
# --batch-size 64              : ОДНАКОВЕ для всіх моделей
# --lr 0.001                   : ОДНАКОВЕ для всіх моделей
# --weight-decay 1e-5          : ОДНАКОВЕ для всіх моделей
# --limit 1000000              : РІВНО 1М судоку (800К train + 200К val)
# 
# 🔧 Архітектурні параметри (специфічні для моделі):
# --hidden-channels 128        : визначено архітектурою
# --num-gnn-layers 8           : визначено архітектурою

# ПРИМІТКА: Gradient clipping (max_norm=1.0) вже в train.py для ВСІХ моделей!

## 📊 Крок 4: Аналіз результатів

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

# Завантаження історії навчання
with open('weights/gnn_history.json', 'r') as f:
    history = json.load(f)

# Підготовка даних
epochs = [h['epoch'] for h in history]
train_loss = [h['train']['loss'] for h in history]
val_loss = [h['val']['loss'] for h in history]
train_acc = [h['train']['cell_accuracy'] for h in history]
val_acc = [h['val']['cell_accuracy'] for h in history]
board_acc = [h['val']['board_accuracy'] for h in history]
lr = [h['lr'] for h in history]

# Візуалізація
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss
axes[0, 0].plot(epochs, train_loss, label='Train Loss', marker='o', alpha=0.7)
axes[0, 0].plot(epochs, val_loss, label='Val Loss', marker='s', alpha=0.7)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('GNN - Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Cell Accuracy
axes[0, 1].plot(epochs, train_acc, label='Train Acc', marker='o', alpha=0.7)
axes[0, 1].plot(epochs, val_acc, label='Val Acc', marker='s', alpha=0.7)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Cell Accuracy')
axes[0, 1].set_title('GNN - Cell Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Board Accuracy
axes[1, 0].plot(epochs, board_acc, label='Board Acc', marker='D', color='green', alpha=0.7)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Board Accuracy')
axes[1, 0].set_title('GNN - Board Accuracy (Complete Puzzles)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Learning Rate
axes[1, 1].plot(epochs, lr, label='Learning Rate', marker='x', color='red', alpha=0.7)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Learning Rate')
axes[1, 1].set_title('GNN - Learning Rate Schedule')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_yscale('log')

plt.tight_layout()
plt.savefig('weights/gnn_training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# Фінальні результати
print("\n" + "="*60)
print("📊 ФІНАЛЬНІ РЕЗУЛЬТАТИ GNN")
print("="*60)
print(f"Best Validation Loss:       {min(val_loss):.4f}")
print(f"Best Cell Accuracy:         {max(val_acc):.2%}")
print(f"Best Board Accuracy:        {max(board_acc):.2%}")
print(f"Final Cell Accuracy:        {val_acc[-1]:.2%}")
print(f"Final Board Accuracy:       {board_acc[-1]:.2%}")
print(f"Final Learning Rate:        {lr[-1]:.6f}")
print("="*60)

# Порівняння з очікуваними результатами
if max(val_acc) >= 0.93:
    print("\n✅ ВІДМІННО! GNN досягла очікуваної точності (93%+)")
elif max(val_acc) >= 0.90:
    print("\n🟡 ДОБРЕ! Можна спробувати збільшити epochs або налаштувати LR")
else:
    print("\n⚠️ Точність нижча за очікувану. Перевірте gradient clipping та LR")

## 🧪 Крок 5: Тестування моделі

In [ ]:
import torch
import numpy as np
from models import GNNModel

# Завантаження моделі
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = GNNModel(hidden_channels=128, num_layers=8).to(device)

checkpoint = torch.load('weights/gnn_best.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Model loaded from epoch {checkpoint['epoch']}")
print(f"   Validation accuracy: {checkpoint['val_accuracy']:.2%}")

# Тестовий пазл (дуже складний)
test_puzzle = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

# Inference
input_tensor = torch.tensor(test_puzzle, dtype=torch.long).unsqueeze(0).to(device)
with torch.no_grad():
    output = model(input_tensor)
    predictions = torch.argmax(output, dim=-1).squeeze().cpu().numpy() + 1
    
    # Confidence
    probs = torch.softmax(output, dim=-1)
    confidence = probs.max(dim=-1)[0].mean().item()

# Виведення результату
print("\n🎯 ТЕСТ ІНФЕРЕНСУ (ДУЖЕ складний пазл - 'Impossible')")
print(f"Confidence: {confidence:.2%}\n")
print("Solved puzzle:")
for row in predictions:
    print(' '.join(map(str, row)))

print("\n💡 GNN теоретично має найкраще впоратися з такими пазлами")
print("   завдяки явному моделюванню Sudoku constraints через граф!")

## 💾 Крок 6: Збереження результатів

In [ ]:
# Створення звіту
report = f"""
╔══════════════════════════════════════════════════════════╗
║           GNN (GAT) - ЗВІТ ПРО НАВЧАННЯ                 ║
╚══════════════════════════════════════════════════════════╝

📊 АРХІТЕКТУРА
├─ Тип:               Graph Attention Network (GAT)
├─ Параметри:         ~300,000
├─ GAT layers:        8 (з 4 attention heads кожен)
├─ Hidden channels:   128
├─ Skip connections:  Yes (8 layers)
├─ Dropout:           0.1
└─ Graph structure:   81 nodes, ~20 edges per node

🎓 НАВЧАННЯ
├─ Epochs:            50
├─ Batch size:        32 (graph processing)
├─ Learning rate:     0.0005 → {lr[-1]:.6f}
├─ Optimizer:         Adam
├─ Scheduler:         ReduceLROnPlateau
├─ Gradient clipping: 1.0 (КРИТИЧНО для GNN)
└─ Device:            {device}

📈 РЕЗУЛЬТАТИ
├─ Best Val Loss:         {min(val_loss):.4f}
├─ Best Cell Accuracy:    {max(val_acc):.2%}
├─ Best Board Accuracy:   {max(board_acc):.2%}
├─ Final Cell Accuracy:   {val_acc[-1]:.2%}
└─ Final Board Accuracy:  {board_acc[-1]:.2%}

💡 ВИСНОВКИ
└─ GNN показує {max(val_acc):.1%} точності на клітинках
   і {max(board_acc):.1%} на повністю вирішених дошках.
   
   🕸️ ТЕОРЕТИЧНО НАЙКРАЩА МОДЕЛЬ для Sudoku:
   • Природньо моделює структуру Sudoku через граф
   • Edges = row/col/box constraints (явне моделювання)
   • Attention механізм вчиться важливості ребер
   • Message passing = логічний inference
   
   ⚠️ Недоліки:
   • Повільніший inference (~25ms vs 5-8ms для CNN)
   • Складніша реалізація
   • Потребує PyTorch Geometric
   • Потребує gradient clipping

📁 ФАЙЛИ
├─ weights/gnn_best.pth               - найкраща модель
├─ weights/gnn_last.pth               - остання модель
├─ weights/gnn_history.json           - історія навчання
└─ weights/gnn_training_curves.png    - графіки

Дата: {history[-1].get('timestamp', 'N/A')}
"""

print(report)

# Збереження звіту
with open('weights/gnn_report.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✅ Звіт збережено у weights/gnn_report.txt")

## ✅ Готово!

GNN (Graph Attention Network) навчена з оптимальними параметрами.  
Теоретично найкраща модель для Sudoku завдяки явному моделюванню constraints!

**Наступний крок:** Порівняйте з іншими моделями:
- `Train_CNN_Baseline.ipynb`
- `Train_CNN_Advanced.ipynb`
- `Train_RNN.ipynb`

**Для дипломної роботи:** GNN - найцікавіша модель для теоретичного аналізу!